# 🦜🔗 LangChain React Agent Demo

In diesem Part schauen wir uns Agenten an. Dabei nutzten wir die neueste Langchain Entwicklung, LangGraph. Mit LangGraph können Agenten als Graphen modelliert werden.

Um die Komplexität gering zu halten, verwenden wir in diesen Beispielen Prebuilt Agent- und Tool-Executors, die von Langchain bereit gestellt werden.

Zuerst installieren wir uns die benötigten Pakete:

In [ ]:
%pip install -r requirements.txt

Wir laden die benötigen Umgebungsvariablen:

In [ ]:
from dotenv import load_dotenv

load_dotenv()

Jetzt definieren wir Tools die unser Agent benutzen kann. In diesem Fall eine Websuche über DuckduckGo. Dadurch erlang der Agent die Fähigkeit Informationen aus dem Internet zu recherchieren.

In [ ]:
from langgraph.prebuilt import ToolExecutor
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

tools = [DuckDuckGoSearchRun()]
tools_executor = ToolExecutor(tools)

Jetzt erstellen wir unseren Agenten. Hierfür benötigen wir ein LLM mit dem der Agent ausgeführt werden soll. 

Wichtig ist hierbei, dass das Sprachmodell auf die Ausführung von Agenten-Logik trainiert sein muss. Wir verwenden in diesem Beispiel die Modelle von OpenAI, da diese optimal für die Ausführung von Agenten-Logik geeignet sind.

Wie vorab erwähnt, nutzen wir einen Prebuilt Agent-Executor, in diesem Beispiel einen der OpenAI Functions ausführen kann.

Nach Ausführung des Agenten bekommen wir die einzelnen Schritte zurück, die der Agent durchgeführt hat.

In [10]:
from langgraph.prebuilt import chat_agent_executor
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from helpers import llm

# app = chat_agent_executor.create_tool_calling_executor(llm, tools_executor)
model = llm.with_config(configurable={"llm": "azure", "temperature": 1})
app = chat_agent_executor.create_function_calling_executor(model, tools)

inputs = {"messages": [HumanMessage(content="Wann wurde Dieter Bohlen geboren? Wann seine Frau? Was ist die Altersdifferenz?")]}

for s in app.stream(inputs):
    print(list(s.values())[0])
    print("-----")


{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Dieter Bohlen Geburtstag"}', 'name': 'duckduckgo_search'}})]}
-----
{'messages': [FunctionMessage(content='Dieter Bohlen. wurde am 7. Februar 1954 geboren. Dieter Bohlen ist ein deutscher Musiker, Sänger, Komponist und Produzent, der mit „Modern Talking" (1984-1987, 1998-2003, u. a. „You\'re My Heart, You\'re My Soul" 1984) bekannt wurde und mit „Deutschland sucht den Superstar" und „Das Supertalent" über fast zwei Jahrzehnte ... Anlässlich seines Jubiläums präsentiert SWR Schlager die wichtigsten Songs des Poptitans, die er für die Teilnehmer von "Deutschland sucht den Superstar" geschrieben hat. Von Alexander Klaws bis Pietro Lombardi, von Steve Benson bis Ramon Roselly - hier sind die Bohlen-Hits im (Pop-)Schlager. Auch Dieter Bohlen hat nun allerdings ein mythisches Alter erreicht. Fit, braun gebrannt wie eh und je, feiert er heute seinen siebzigsten Geburtstag. Wäre es da nicht an der 

In diesem Beispiel führen wir den zuvor definierten Agenten nochmals aus und streamen diesmal allerdings nur die finale Antwort des Agentenlauf raus. Dabei wird die Antwort Token für Token ausgegeben.

In [11]:
model = ChatOpenAI(streaming=True)
app = chat_agent_executor.create_function_calling_executor(model, tools)

    
async for output in app.astream_log(inputs, include_types=["llm"]):
    for op in output.ops:
        if op["path"] == "/streamed_output/-":
            # this is the output from .stream()
            ...
        elif op["path"].startswith("/logs/") and op["path"].endswith(
            "/streamed_output/-"
        ):
            # because we chose to only include LLMs, these are LLM tokens
            token_content = op["value"].content
            if token_content:
                print(op["value"].content, end="", flush=True)

Dieter Bohlen wurde am 7. Februar 1954 geboren. Seine Frau, Carina Bohlen, wurde im Februar 1993 geboren. Die Altersdifferenz zwischen Dieter Bohlen und Carina Bohlen beträgt etwa 39 Jahre.

TODO: Jetzt noch mit dem Tool executor, nicht mit function executor